In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
!pip install torch_geometric

In [37]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


In [38]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


# Requirements

In [39]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

In [40]:
logger = logging.getLogger(__name__)

In [41]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [42]:
from src.optimization import MatrixFeatureBasedMDAClassifierTrainer, MatrixFeatureBasedMDAClassifierTester
from src.config import SimpleClassifierConfig
from src.models import DummyFeatureBasedMDAClassifier, DummyFeatureBasedMDAClassifierFactory
from src.data import MicrobeDiseaseAssociationData, MicrobeDiseaseAssociationTrainTestSpliter
from src.features import get_associations, get_entities
from src.utils import train_test_sampler
from base import cross_validation, OptimizerConfig


# Classification

## Data

In [43]:
associations = get_associations()

train_indices, test_indices = train_test_sampler(associations.shape[0], 0.7)

data = MicrobeDiseaseAssociationData(associations)

train_data = MicrobeDiseaseAssociationData(associations.iloc[train_indices])
test_data = MicrobeDiseaseAssociationData(associations.iloc[test_indices])

## Classifier

In [44]:
microbe_ids = get_entities().loc[get_entities()['type'] == 'Microbe']['id'].tolist()
disease_ids = get_entities().loc[get_entities()['type'] == 'Disease']['id'].tolist()

In [45]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = len(microbe_ids) + len(disease_ids)
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.1

In [46]:
mda_classifier = DummyFeatureBasedMDAClassifier(simple_classifier_config, microbe_ids=microbe_ids, disease_ids=disease_ids)

2024-02-10 19:35:18,875 [INFO] Initializing MDFeatureBasedMDAClassifier with model : simple classifier
2024-02-10 19:35:18,881 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-10 19:35:18,887 [INFO] Initial SimpleMLP with 10824 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-10 19:35:18,901 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 19:35:18,903 [INFO] Initializing MatrixDummyFeatureExtractor


## Optimizer

In [47]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [48]:
train_result = MatrixFeatureBasedMDAClassifierTrainer().train(model=mda_classifier, data=train_data,config=classifier_optimizer_config)

2024-02-10 19:35:18,955 [INFO] Call Training with adam optimizer
2024-02-10 19:35:18,980 [INFO] Calling build with associations :      disease  microbe  increased
589    48777     9788          0
212    50863    50698          1
202    30861    59219          1
387    43621    20492          1
723    30861    25804          0
..       ...      ...        ...
403    12403    13682          1
864    12403    64598          0
201    12403    26183          1
143    43621    19319          1
531    64642    59489          0

[628 rows x 3 columns]
2024-02-10 19:35:19,076 [INFO] interaction matrix with shape (5179, 5645) has built
2024-02-10 19:35:19,566 [INFO] mask matrix with shape (5179, 5645) has built. This matrix shows not non elements.
2024-02-10 19:35:21,739 [INFO] interaction has been imputed to delete nans
2024-02-10 19:35:22,095 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 10824]) and y shape : torch.Size([628, 1])
2024-02-10 19:35:22,097 [INFO] Running S

### Test

In [49]:
test_result = MatrixFeatureBasedMDAClassifierTester().test(model=mda_classifier, data=test_data, config=classifier_optimizer_config)

2024-02-10 19:35:32,936 [INFO] Call Testing with adam optimizer
2024-02-10 19:35:33,207 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 10824]) and y shape : torch.Size([270, 1])
2024-02-10 19:35:33,220 [INFO] Running Simple Tester with config : adam optimizer
2024-02-10 19:35:33,226 [INFO] moving data and model to cpu
2024-02-10 19:35:33,301 [INFO] Result on Test Data : {'AUC': 0.5, 'ACC': 0.48148148148148145, 'F1 Score': 0.325, 'AUPR': 0, 'Loss': 0.9917609559165107}


In [50]:
test_result.get_result()

{'AUC': 0.5,
 'ACC': 0.48148148148148145,
 'F1 Score': 0.325,
 'AUPR': 0,
 'Loss': 0.9917609559165107}

## Cross Validation

In [51]:
trainer = MatrixFeatureBasedMDAClassifierTrainer()
tester = MatrixFeatureBasedMDAClassifierTester()
factory = DummyFeatureBasedMDAClassifierFactory(model_config=simple_classifier_config, microbe_ids=microbe_ids, disease_ids=disease_ids)
spliter = MicrobeDiseaseAssociationTrainTestSpliter(data.associations)
cross_validation(k=5, data_size=data.associations.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-10 19:35:33,362 [INFO] Initializing DummyFeatureBasedMDAClassifierFactory
2024-02-10 19:35:33,372 [INFO] Initializing MicrobeDiseaseAssociationTrainTestSpliter
2024-02-10 19:35:33,380 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-10 19:35:33,384 [INFO] ---- Fold 1 ----
2024-02-10 19:35:33,392 [INFO] Initializing MDFeatureBasedMDAClassifier with model : simple classifier
2024-02-10 19:35:33,397 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-10 19:35:33,399 [INFO] Initial SimpleMLP with 10824 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-10 19:35:33,417 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 19:35:33,421 [INFO] Initializing MatrixDummyFeatureExtractor
2024-02-10 19:35:33,423 [INFO] Call Training with adam optimizer
2024-02-10 19:35:33,430 [INFO] Calling build with associations :      disease  microbe  increased
0      50863    33211          1
